In [3]:
#!pip install torch==1.5.0 torchvision==0.6.0
!pip install transformers seqeval==0.0.12 &> /dev/null
#!pip install tensorflow==2.2.0 &> /dev/null

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import json
import pandas as pd
import logging
import re

import json
import numpy as np
import torch
import pickle
from tqdm import trange
from tqdm import tqdm_notebook as tqdm
from transformers import BertForTokenClassification, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

#Model loading
MAX_LEN = 500
max_len = 500
EPOCHS = 6 
MAX_GRAD_NORM = 1.0

#gpu
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model path
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/Models/BERT/'
TOKENIZER = BertTokenizerFast('/content/drive/MyDrive/Colab Notebooks/Models/BERT/vocab.txt', lowercase=True)


import pandas as pd 
#df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/datasets/pre_trained.json", lines = True) #701 rows
df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/datasets/RP/NER_RESUME.json", lines = True) #300 rows

#/content/drive/MyDrive/Colab Notebooks/datasets/RP/300-without extras.json
df.drop(['extras'], axis=1)

#entity list
tags_vals = ['Email Address', 'Links', 'Skills', 'Graduation Year', 'College Name', 'Degree', 'Companies worked at', 'Location', 'Name', 'Designation', 'projects', 'Years of Experience', 'Can Relocate to', 'UNKNOWN', 'Rewards and Achievements', 'Address', 'University', 'Relocate to', 'Certifications', 'state', 'links', 'College', 'training', 'des', 'abc']


In [5]:
import json
import pickle
import re

def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    print("Cleaned trailing spaces")

    return cleaned_data



def for_spacy(input_file):
    count = 0
    class_labels = list()
    training_data = []
    lines = []
    with open(input_file, 'r')  as f:
        lines = f.readlines()
    
    for line in lines:
        data = json.loads(line)
        entities = []
        if  data['annotation'] == None:
            continue
        text = data['content']
        count += 1
        for annotation in data['annotation']:
            point = annotation['points'][0]
            labels = annotation['label']
            
            for lb in labels:
                if lb not in class_labels:
                    class_labels.append(lb)
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                entities.append((point['start'], point['end'] + 1 ,label))


        training_data.append((text, {"entities" : entities}))
    training_data = trim_entity_spans(training_data)
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [5]:
#train_data['entities']

In [6]:
#data = trim_entity_spans(convert_dataturks_to_spacy("/content/drive/MyDrive/Colab Notebooks/datasets/pre_trained.json"))
data = for_spacy("/content/drive/MyDrive/Colab Notebooks/datasets/RP/NER_RESUME.json") #300

In [7]:
data[0]

["Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n

In [9]:
# Changing data to appropriate format so as to feed it to the model
from tqdm import tqdm_notebook as tqdm
cleanedDF = pd.DataFrame(columns=["setences_cleaned"])
sum1 = 0
for i in tqdm(range(len(data))):
    start = 0
    emptyList = ["Empty"] * len(data[i][0].split())
    numberOfWords = 0
    lenOfString = len(data[i][0])
    strData = data[i][0]
    strDictData = data[i][1]
    lastIndexOfSpace = strData.rfind(' ')
    for i in range(lenOfString):
        if (strData[i]==" " and strData[i+1]!=" "):
            for k,v in strDictData.items():
                for j in range(len(v)):
                    entList = v[len(v)-j-1]
                    if (start>=int(entList[0]) and i<=int(entList[1])):
                        emptyList[numberOfWords] = entList[2]
                        break
                    else:
                        continue
            start = i + 1  
            numberOfWords += 1
        if (i == lastIndexOfSpace):
            for j in range(len(v)):
                    entList = v[len(v)-j-1]
                    if (lastIndexOfSpace>=int(entList[0]) and lenOfString<=int(entList[1])):
                        emptyList[numberOfWords] = entList[2]
                        numberOfWords += 1
    cleanedDF = cleanedDF.append(pd.Series([emptyList],  index=cleanedDF.columns ), ignore_index=True )
    sum1 = sum1 + numberOfWords

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/220 [00:00<?, ?it/s]

In [ ]:
cleanedDF['setences_cleaned'][0]

In [7]:
def get_label(offset, labels): #for 0 labels which are increased by padding are tagged as 0
    if offset[0] == 0 and offset[1] == 0:
        return 'O'
    for label in labels:
        if offset[1] >= label[0] and offset[0] <= label[1]:
            return label[2]
    return 'O'

In [8]:
tag2idx = {t: i for i, t in enumerate(tags_vals)} # tags as {keys} and number as {value}
idx2tag = {i:t for i, t in enumerate(tags_vals)} #numbering of tags{value} from 0{key} #19

### 1 - process_resume

In [9]:
def process_resume(data, tokenizer, tag2idx, max_len, is_test=False): 
#is_test=False
    tok = tokenizer.encode_plus(data[0], max_length=max_len, return_offsets_mapping=True) #tokeniztion code - tok
    '''(change here)'''

    curr_sent = {'orig_labels':[], 'labels': []} #current sentence - orgin_label & labels - DICT

    padding_length = max_len - len(tok['input_ids']) 
    if not is_test:#getting data
        labels = data[1]['entities']
        labels.reverse() #reversing
        for off in tok['offset_mapping']:
            label = get_label(off, labels)
            #print(label)
            curr_sent['orig_labels'].append(label)
            #print(curr_sent['orig_labels'])
            curr_sent['labels'].append(tag2idx[label])
        curr_sent['labels'] = curr_sent['labels'] + ([0] * padding_length)
        print(curr_sent['labels'])
        print(curr_sent['orig_labels'])

    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length) #current sentence labeling - tokenzing
    #print(curr_sent['input_ids'])
    curr_sent['token_type_ids'] = tok['token_type_ids'] + ([0] * padding_length)
    #print(curr_sent['token_type_ids'])
    curr_sent['attention_mask'] = tok['attention_mask'] + ([0] * padding_length) #dividing intp 3 current sentences - DICT
    #print(curr_sent['attention_mask'])
    return curr_sent

### 2 - CLASS - ResumeDataset

In [10]:
class ResumeDataset(Dataset):
    def __init__(self, resume, tokenizer, tag2idx, max_len, is_test=False): #init def
        self.resume = resume
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.tag2idx = tag2idx
        self.max_len = max_len
    
    def __len__(self):
        return len(self.resume)
    
    def __getitem__(self, idx): #getiing a item and iterating
        data = process_resume(self.resume[idx], self.tokenizer, self.tag2idx, self.max_len, self.is_test)
        #data = process_resume(self.resume[idx], self.tokenizer,self.tag2idx,self.max_len)
        return {
            'input_ids': torch.tensor(data['input_ids'], dtype=torch.long),
            'token_type_ids': torch.tensor(data['token_type_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(data['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(data['labels'], dtype=torch.long),
            'orig_label': data['orig_labels'] #keep same no need to convert to tensor
        }


#dividing the dataset #corrected
total = len(data)
#train_data, val_data = data[:560], data[560:] 
train_data, val_data = data[:180], data[180:] #300

In [11]:
train_data[1][1]

{'entities': [[1155, 1199, 'Email Address'],
  [743, 1141, 'Skills'],
  [729, 733, 'Graduation Year'],
  [675, 702, 'College Name'],
  [631, 673, 'Degree'],
  [625, 629, 'Graduation Year'],
  [614, 623, 'College Name'],
  [606, 612, 'Degree'],
  [104, 148, 'Email Address'],
  [62, 68, 'Location'],
  [0, 14, 'Name']]}

In [12]:
train_d = ResumeDataset(train_data, TOKENIZER, tag2idx, MAX_LEN) #calling class here
val_d = ResumeDataset(val_data, TOKENIZER, tag2idx, MAX_LEN)

In [13]:
train_d #attention_mask is a dict-key and tensor is the value

In [14]:
train_sampler = RandomSampler(train_d)
train_dl = DataLoader(train_d, sampler=train_sampler, batch_size=8,collate_fn=lambda x: x )
val_dl = DataLoader(val_d, batch_size=8)

In [15]:
def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"] #no decay
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

    return optimizer_grouped_parameters

In [16]:
def get_special_tokens(tokenizer, tag2idx): #?
    vocab = tokenizer.get_vocab()
    pad_tok = vocab["[PAD]"]
    sep_tok = vocab["[SEP]"]
    cls_tok = vocab["[CLS]"]
    o_lab = tag2idx["O"]

    return pad_tok, sep_tok, cls_tok, o_lab


def annot_confusion_matrix(valid_tags, pred_tags):

    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    # Create header from unique tags
    header = sorted(list(set(valid_tags + pred_tags)))

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t\t\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


#flat accuracy
def flat_accuracy(valid_tags, pred_tags):#accuracy
    return (np.array(valid_tags) == np.array(pred_tags)).mean()

In [17]:
#Model Load
model = BertForTokenClassification.from_pretrained(MODEL_PATH, num_labels=len(tag2idx)) #num label
model.to(DEVICE);
optimizer_grouped_parameters = get_hyperparameters(model, True)
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Models/BERT/ were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

### Train

In [24]:
def train_and_save_model(
    model,
    tokenizer,
    optimizer,
    epochs,
    idx2tag,
    tag2idx,
    max_grad_norm,
    device,
    train_dataloader,
    valid_dataloader
):

    pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, tag2idx)
    
    #Empty Lists
    epoch = 0
    val_acc=[]
    ep=[]
    val_loss=[]

    for _ in trange(epochs, desc="Epoch"):
        epoch += 1 #+1

        # Training loop
        print("Starting training loop. Hope it works !!!")
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        for step, batch in enumerate(train_dataloader):
            # Add batch to gpu
            
            #batch = tuple(t.to(device) for t in batch)
            #b_input_ids, b_input_mask, b_labels = batch
            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            #b_input_ids, b_input_mask, b_labels = batch[step]['input_ids'], batch[step]['attention_mask'], batch[step]['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss, tr_logits = outputs[:2]

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            tr_logits = tr_logits.cpu().detach().numpy()
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1)
            tr_batch_labels = tr_label_ids.to("cpu").numpy()
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)

            # Compute training accuracy
            tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=max_grad_norm
            )

            # Update parameters
            optimizer.step()
            model.zero_grad()

        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"Train loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")
        
        
        """
        Validation loop
        """ 
        print("Starting validation loop.")

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []

        for batch in valid_dataloader:
            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            #b_input_ids, b_input_mask, b_labels = batch[step]['input_ids'], batch[step]['attention_mask'], batch[step]['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                tmp_eval_loss, logits = outputs[:2]

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            logits = logits.cpu().detach().numpy()
            label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            val_batch_preds = np.argmax(logits[preds_mask.squeeze()], axis=1)
            val_batch_labels = label_ids.to("cpu").numpy()
            predictions.extend(val_batch_preds)
            true_labels.extend(val_batch_labels)

            tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)

            eval_loss += tmp_eval_loss.mean().item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1

        # Evaluate loss, acc, conf. matrix, and class. report on devset
        pred_tags = [idx2tag[i] for i in predictions]
        valid_tags = [idx2tag[i] for i in true_labels]
        cl_report = classification_report([valid_tags], [pred_tags])
        conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_steps

        # Report metrics
        print(f"Validation loss: {eval_loss}")
        print(f"Validation Accuracy: {eval_accuracy}")
        print(f"Classification Report:\n {cl_report}")
        print(f"Confusion Matrix:\n {conf_mat}")
        val_acc.append(eval_accuracy)
        ep.append(epoch)
        val_loss.append(eval_loss)
    
    plt.plot(ep, val_acc, 'g', label='Validation accuracy')
    #plt.plot(ep, val_loss, 'b', label='Validation loss')
    plt.title('Validation acuuracy ')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()    

In [25]:
train_and_save_model(
    model, 
    TOKENIZER, 
    optimizer, 
    EPOCHS, 
    idx2tag, 
    tag2idx, 
    MAX_GRAD_NORM,
    DEVICE, 
    train_dl, 
    val_dl
)

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Starting training loop. Hope it works !!!
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 11, 11, 11, 11, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

TypeError: ignored

In [ ]:
resume = data[41][0]
entities = predict(model, TOKENIZER, idx2tag, tag2idx, DEVICE, resume)

In [ ]:
STATE_DICT = torch.load('../working/model_e10.tar', map_location=DEVICE)

In [ ]:
# def process_resume(text, tokenizer, max_len,tag2idx):
#     tok = tokenizer.encode_plus(text, max_length=max_len, return_offsets_mapping=True)
    
#     curr_sent = dict()
    
#     padding_length = max_len - len(tok['input_ids']) #padding lenght
        
#     curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length) #current sentence
#     print(curr_sent['input_ids'])
#     curr_sent['token_type_ids'] = tok['token_type_ids'] + ([0] * padding_length)
#     print(curr_sent['token_type_ids'])
#     curr_sent['attention_mask'] = tok['attention_mask'] + ([0] * padding_length)
#     print(curr_sent['attention_mask'])
    
#     final_data = {
#         'input_ids': torch.tensor(curr_sent['input_ids'], dtype=torch.long),
#         'token_type_ids': torch.tensor(curr_sent['token_type_ids'], dtype=torch.long),
#         'attention_mask': torch.tensor(curr_sent['attention_mask'], dtype=torch.long),
#         'offset_mapping': tok['offset_mapping']
#     }
    
#     return final_data

'''

# def predict(model, tokenizer, idx2tag, tag2idx, device, test_resume):
#     model.eval()
#     data = process_resume(test_resume, tokenizer, MAX_LEN)
#     input_ids, input_mask = data['input_ids'].unsqueeze(0), data['attention_mask'].unsqueeze(0)
#     labels = torch.tensor([1] * input_ids.size(0), dtype=torch.long).unsqueeze(0)
#     with torch.no_grad():
#         outputs = model(
#             input_ids,
#             token_type_ids=None,
#             attention_mask=input_mask,
#             labels=labels,
#         )
#         tmp_eval_loss, logits = outputs[:2]
    
#     logits = logits.cpu().detach().numpy()
#     label_ids = np.argmax(logits, axis=2)
    
#     entities = []
#     for label_id, offset in zip(label_ids[0], data['offset_mapping']):
#         curr_id = idx2tag[label_id]
#         curr_start = offset[0]
#         curr_end = offset[1]
#         if curr_id != 'O':
#             if len(entities) > 0 and entities[-1]['entity'] == curr_id and curr_start - entities[-1]['end'] in [0, 1]:
#                 entities[-1]['end'] = curr_end
#             else:
#                 entities.append({'entity': curr_id, 'start': curr_start, 'end':curr_end})
#     for ent in entities:
#         ent['text'] = test_resume[ent['start']:ent['end']]
#     return entities